# Script to create vector tiles for direct usage in LPVIS
- Input: AMA produced shapefiles with parcels and physical blocks downloaded from https://www.data.gv.at/katalog/dataset/invekos-schlaege-oesterreich/resource/26e5b6c4-6e47-45d3-ac65-728c631fd515 and https://www.data.gv.at/katalog/dataset/invekos-referenzen-oesterreich/resource/e883c5d0-ea79-44be-b789-d8bd6a3389e3, bounding box
- Output: zoom 14-16 uncompressed pbf vector tiles used in LPvis in two folders wrapped in .tar.gz
These should be manually moved to LPvis local folder and committed to GitHub.

In [ ]:
from shapely.geometry import Polygon
import re, time, progressbar
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
import subprocess, os
# Needs `tippecanoe` installed

In [ ]:
# Modify input/output/bbox as necessary
bbox = [15.4, 48.54, 15.96, 48.92]
input_parcels_file = '/home/lubomir/lpis-data-at/invekos_schlaege_polygon.shp'
input_blocks_file = '/home/lubomir/lpis-data-at/invekos_referenzen_polygon.shp'
output_parcels_file = 'data/agricultural_parcels.gpkg'
output_physical_blocks_file = 'data/physical_blocks.gpkg'
parcels_directory = 'agricultural_parcels'
blocks_directory = 'physical_blocks'


In [ ]:
print('Reading data parcels')
gdf = gpd.read_file(input_parcels_file)

In [ ]:
inProj = Proj('epsg:4326')
outProj = Proj(gdf.crs['init'])
# switched order of x,y on purpose
y1_t,x1_t = transform(inProj,outProj,bbox[1],bbox[0])
y2_t,x2_t = transform(inProj,outProj,bbox[3],bbox[2])

In [ ]:
print('Performing spatial subset')
spatial_subset = gdf.cx[x1_t:x2_t, y1_t:y2_t]
print('Converting to mercator for tippecanoe')
spatial_subset_m = spatial_subset.to_crs('epsg:4326')

In [ ]:
# variable substitution / classification group merging when creating vector tiles for eurodatacube/LPVIS 
def replace_umlauts(s):
    s = re.sub(r"(\b|[^AEIOU])AE", r'\1Ä', s)
    s = re.sub(r"(\b|[^AEIOU])OE", r'\1Ö', s)
    s = re.sub(r"(\b|[^AEIOU])UE", r'\1Ü', s)
    return s
print('Merging with classification groups from xls file')
renamed = spatial_subset_m.rename(columns={'SNAR_BEZEI':'CT'})

lut = pd.read_excel('data/2_LUT_list_Version2_20190724.xlsx')
lut.CT = lut.CT.apply(replace_umlauts)

merged = renamed.merge(lut, how='left', on='CT')

final_data = merged[['ID', 'Ctnum','FS_KENNUNG','CTnumL4A', 'CT', 'SL_FLAECHE', 'geometry']]
try:
    print('Deleting output gpkg file, to not append.')
    os.unlink(output_parcels_file)
except:
    pass
final_data.to_file(output_parcels_file, driver="GPKG")
print('Data successfully written!')

In [ ]:
print('Creating vector tiles')

fio = subprocess.Popen(('fio', 'cat', output_parcels_file), stdout=subprocess.PIPE)
tippecanoe = subprocess.check_output(('tippecanoe', '--output-to-directory=%s' % parcels_directory, 
                              '--drop-densest-as-needed', '--read-parallel', '--minimum-zoom=14', 
                              '--maximum-zoom=16', '-l', 'agricultural_parcels', '-f', '--no-tile-compression'), stdin=fio.stdout)
fio.wait()

In [ ]:
print('Reading data physical')
gdf = gpd.read_file(input_blocks_file)

In [ ]:
print('Performing spatial subset')
spatial_subset = gdf.cx[x1_t:x2_t, y1_t:y2_t]
print('Converting to mercator for tippecanoe')
spatial_subset_m = spatial_subset.to_crs('epsg:4326')

In [ ]:
try:
    print('Deleting output gpkg file, to not append.')
    os.unlink(output_physical_blocks_file)
except:
    pass
spatial_subset_m.to_file(output_physical_blocks_file, driver="GPKG")
print('Data successfully written!')

In [ ]:
print('Creating vector tiles')

fio = subprocess.Popen(('fio', 'cat', output_physical_blocks_file), stdout=subprocess.PIPE)
tippecanoe = subprocess.check_output(('tippecanoe', '--output-to-directory=%s' % blocks_directory, 
                              '--drop-densest-as-needed', '--read-parallel', '--minimum-zoom=14', 
                              '--maximum-zoom=16', '-l', 'physical_blocks', '-f', '--no-tile-compression'), stdin=fio.stdout)
fio.wait()

In [ ]:
# for LPvis repository, the files are stored as compressed archive for storage reasons
for tile_dir in [parcels_directory, blocks_directory]:
    tar = subprocess.run(['tar', '-czvf', '%s.tar.gz' % tile_dir, tile_dir])